# Simple Image Classifier Using Tensorflow

Here, we will create an image classifier with Tensorflow that will be able to classify images of your choice. This example utilizes the [Inception v3](https://github.com/tensorflow/models/tree/master/inception) architecure model. Inception is a pretrained [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) model trained on over a hundred thousand of [ImageNet](http://www.image-net.org/) images, classifying them into 1000 categories.

In order to get the model to be able to identify a custom image, we must apply [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). This is where we apply the learning from a previous training session to a new training session.

The magic of transfer learning is that lower layers that have been trained to distinguish between some objects can be reused for many recognition tasks without any alteration.

To do the transfer learning, run the retrain script included in the Tensorflow repository.

````
python tensorflow/examples/image_retraining/retrain.py \
--bottleneck_dir=/tf_demo/bottlenecks \
--how_many_training_steps 1000 \
--model_dir=/tf_demo/inception \
--output_graph=/tf_demo/retrained_graph.pb \
--output_labels=/tf_demo/retrained_labels.txt \
--image_dir /tf_demo/data
````

The ***bottleneck_dir*** will be used to cache the outputs of the lower layers on disk so they don’t have to repeatedly be recalculated. 'Bottleneck' is an informal term often used for the layer just before the final output layer that actually does the classification.

We will run this example for 1000 iterations. This amount can be experimented as the improvement in accuracy has diminishing returns the longer you train for.

The ***model_dir*** flag asks us where to store the trained model.

The ***output_graph*** option is where the script will write out a version of the Inception v3 neural network with a final layer retrained to our categories. 

The ***output_labels*** will be the file where the labels are stored. These labels are the same as the image folder names.

This will take a while, but when it is complete, you will now have a retrained model.

### Detector Script

We will now write a script that will use the retrained model to detect if an image contains the subject we trained it on. Also, the repo of this notebook contains a simple python script where you can pass in your image and model directory to classify your own image using the retrained model.

In [1]:
import tensorflow as tf

In [2]:
# Change these to match your paths
# The image we want to test
image_path = '/Users/paulvaneck/tensorflow-demo/test-images/batman1.jpg'

# Directory containing graph file and labels file
model = '/Users/paulvaneck/tensorflow-demo/hero-model/'

In [3]:
# Read in the image_data
image_data = tf.gfile.FastGFile(image_path, 'rb').read()

In [4]:
# Loads label file to get all labels
label_list = [line.rstrip() for line 
                   in tf.gfile.GFile(model + "retrained_labels.txt")]

In [5]:
# Get the retrained model from the saved graph file.
with tf.gfile.FastGFile(model + "retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()

    # Parse the graph.
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [6]:
with tf.Session() as sess:
    # Get the softmax tensor from the last layer of our model.
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

    # Feed the image_data as input to the graph
    # Map input data into probabilites with softmax
    predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})

    # Sort to show labels of predictions by confidence
    sorted_nodes = predictions[0].argsort()[-len(predictions[0]):][::-1]

    # For every prediction, print out the label and score.
    for node in sorted_nodes:
        label = label_list[node]
        score = predictions[0][node]
        print('%s (score = %.5f)' % (label, score))

batman (score = 0.99923)
superman (score = 0.00077)


#### Additional Information

https://www.tensorflow.org/tutorials/image_retraining
